In [2]:
import pandas as pd
import keras
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = 9,6

from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings(action='ignore')

from generate_custome_models import generate_model1, generate_model2, generate_model3
from generate_pretrained_models import generate_resnet_model, generate_vgg_model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from print_utils import print_model_summaries_side_by_side

# Data loading and preparation

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES=10

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [5]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

# Fitting the custom deep learning models

In [6]:
LR=1e-3 
EPOCHS=100  
PATIENCE=10
MIN_DELTA=0.01
early_stop = EarlyStopping(monitor='val_accuracy', patience=PATIENCE, min_delta=MIN_DELTA)

model1=generate_model1(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model2=generate_model2(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
model3=generate_model3(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

print_model_summaries_side_by_side([model1, model2, model3])



Model: "sequential"                                                              | Model: "sequential_1"                                                            | Model: "sequential_2"                                                           
_________________________________________________________________                | _________________________________________________________________                | _________________________________________________________________               
 Layer (type)                Output Shape              Param #                   |  Layer (type)                Output Shape              Param #                   |  Layer (type)                Output Shape              Param #                  
=================================================================                | =================================================================                | =================================================================               
 conv2d (C

# Fitting pre-trained arch

In [7]:
resnet_model=generate_resnet_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
vgg_model=generate_vgg_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)


print_model_summaries_side_by_side([resnet_model, vgg_model])

Model: "sequential_3"                                                            | Model: "sequential_4"                                                           
_________________________________________________________________                | _________________________________________________________________               
 Layer (type)                Output Shape              Param #                   |  Layer (type)                Output Shape              Param #                  
=================================================================                | =================================================================               
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712                  |  vgg16 (Functional)          (None, 1, 1, 512)         14714688                 
                                                                                 |                                                                                 
 flatten_3 (Flat